In [1]:
import os
from gmsPython.gmsWrite.gmsWrite import *

*Load test database with symbols:*

In [2]:
work_folder = os.path.join(os.getcwd(),'workFolder')
data_folder = os.path.join(os.getcwd(), 'test')
db = gpyDB.GpyDB(db = os.path.join(data_folder, 'test_size1000.gdx'), ws = work_folder)

## 1. Writing gpy symbols:

We can write ```gpy``` symbols to gams syntax using the ```Syms.gpy``` method:

*Write without conditions:*

In [3]:
for k in db:
    print(Syms.gpy(k))

alias_set
alias_map2
i
j
alias_[alias_set,alias_map2]
map[i,j]
var[i,j]
var1d[i]
param[i,j]
pscalar
subset[j]
sclr


### 1.1. Adding conditionals

We add conditions by specifying the kwarg ```c```. It follows the syntax from ```pyDatabases.adj.rc_AdjGpy``` which means that the same syntax that make adjustments to the python object can be translated into GAMS syntax.

#### Use ```gpy``` symbol as condition:

In [4]:
Syms.gpy(s = db['var'], c = db['map']) # condition on map

'var[i,j]$(map[i,j])'

In [5]:
Syms.gpy(s = db['var'], c = db['subset']) # condition on subset

'var[i,j]$(subset[j])'

*Note: All the statements used to write GAMS code here can also be used to subset the symbols in Python. For instance:*

In [6]:
pyDatabases.adj.rc_AdjGpy(db['var'], c = db['subset']).vals

i  j  
1  1      10.0
   0      10.0
   2      10.0
   3      10.0
   4      10.0
          ... 
   495    10.0
   496    10.0
   497    10.0
   498    10.0
   499    10.0
Name: var, Length: 500, dtype: float64

#### Use ```dict``` as condition:

If we pass a dict as a condition, the text that is added as condition will be run through the ```Syms.gpy``` method itself. Thus, as we will see later, we may e.g. suggest that we only condition on the subset where the domain $j$ is lagged:

In [7]:
Syms.gpy(s = db['var'], 
          c = {'s': db['subset'], 'lag': {'j':-1}}) 

'var[i,j]$(subset[j-1])'

*Note: If we pass lag as an integer instead of a dictionary, this will apply to **all** domains:* 

In [8]:
Syms.gpy(s = db['var'], 
          c = {'s': db['subset'], 'lag': -1}) # this still works

'var[i,j]$(subset[j-1])'

In [9]:
Syms.gpy(s = db['var'],
          c = {'s': db['map'], 'lag': -1}) # this lags all domains in the condition, both i and j

'var[i,j]$(map[i-1,j-1])'

#### Use ```tuple``` as condition:

We use tuples to pass nested conditions or add negation. For instance:

In [10]:
Syms.gpy(s = db['var'],
          c = ('not', db['subset'])) # negation of condition

'var[i,j]$(( not (subset[j])))'

In [11]:
Syms.gpy(s = db['var'],
          c = ('or', [db['subset'], db['map'], {'s': db['subset'], 'lag': -1}])) # add 'or' between conditions in list

'var[i,j]$((subset[j] or map[i,j] or subset[j-1]))'

In [12]:
Syms.gpy(s = db['var'],
          c = ('and', [db['subset'], db['map'], {'s': db['subset'], 'lag':-1}])) # add 'and' between conditions

'var[i,j]$((subset[j] and map[i,j] and subset[j-1]))'

*Note: Nested conditions can also nest other condtions - the methods work iteratively, so go nuts:*

In [13]:
c = ('and', [db['subset'], 
             ('or', [{'s': db['subset'], 'lag':-1}, db['map']])
            ]
    )
Syms.gpy(s = db['var'], c = c)

'var[i,j]$((subset[j] and (subset[j-1] or map[i,j])))'

#### Use ```str``` as condition:

The final option is simply to Syms. the condition yourself. 

*Note: This is the one option that cannot be passed to the method ```rc_AdjGpy```.*

In [14]:
Syms.gpy(s = db['var'], c = 'subset[jj]')

'var[i,j]$(subset[jj])'

### 1.2. Alias domains

The alias kwarg allows us to use an aliased domain. For instance, the database has aliased $j$ with $jj$. The write method uses this to adjust domain names:

In [15]:
Syms.gpy(s = db['var'], alias = {'j':'jj'})

'var[i,jj]'

*Note: If the alias is used to write a set (I know this is a weird use-case)*

In [16]:
Syms.gpy(s = db['j'], alias = {'j':'jj'})

'jj'

### 1.3. Lag domains

Domains are lagged by adding what levels (adding a dictionary) or on all levels (adding an int):

In [17]:
Syms.gpy(s = db['var'], lag = {'i':'+1', 'j':-1})

'var[i+1,j-1]'

### 1.4. Specify attribute ```l```

Finally, for variables we may want to set/get a particular attribute of the symbol. In this case, we may add e.g. ```l=.l``` to indicate this:

In [18]:
Syms.gpy(s = db['var'], l = '.l')

'var.l[i,j]'

## 2. Write from GpyDB

Write declare and load statements:

In [19]:
db.mergeInternal() # write gpy symbols to GamsDatabase

Create text for declaring and loading symbols from the database:

In [20]:
decl = FromDB.declare(db)
load = FromDB.load(db, gdx = db.name)

Create a GamsJob containing the declaring and loading:

In [21]:
job = db.ws.add_job_from_string(decl+load)

Run job with the database as an option:

In [22]:
job.run(databases = db.database)

*Note: Using the databases option means exporting to a gdx file in the working directory - there is no "direct" link to the GamsDatabase instance. This means that we can just as easily write the load statement and point to a different database name, then export the database with this name, and run the file **without** the database option. That is, the following syntax gives identical results:*

In [23]:
load_newName = FromDB.load(db, gdx = 'newname')
db.database.export('newname')
db.ws.add_job_from_string(decl+load_newName).run() # run without the database option.

*Note: One way to set lower/upper bounds on a variable is to include these directly in the database by specifying the ```self.defaultAttrs```, ```self.np```, or ```self.df``` attribute of the relevant gdx symbol:*

In [24]:
db['var'].defaultAttrs['lower'] = 0 # set the default lower bound to 0 - used to create .np 

## 3. Write from GmsPy instance

We use the ```GmsPy``` instance to define model settings.